# Analyzing Scenario Results

In [1]:
from datetime import datetime, timedelta
from src.configs import (
    ADDRESSES,
    TOKEN_DTOs,
    get_scenario_config,
    get_price_config,
    CRVUSD_DTO,
)
import pickle
import pandas as pd
import json
import os
import numpy as np
from crvusdsim.pool import get  # type: ignore
from copy import deepcopy
from src.sim.scenario import Scenario

pd.set_option("display.max_columns", None)

%load_ext autoreload
%autoreload 2

In [133]:
# Number of buckets to split rows into
def mom(X, b):
    row_indices = np.arange(X.shape[0])
    np.random.shuffle(row_indices)
    buckets = np.array_split(X[row_indices], b, axis=0)
    bucket_means = [np.mean(x, axis=0) for x in buckets]
    median = np.median(np.array(bucket_means), axis=0)
    return median


def mean_mom(X):
    b = int(np.sqrt(X.shape[0]))
    return np.mean([mom(X, b) for _ in range(1000)])


def get_results(scenario: str, resampling: bool = False) -> list:
    """
    Run a simulation and return the results.
    """
    results = []
    dir_ = f"../result/{scenario}/sfrxeth_wbtc_weth_wsteth/"
    files = [os.path.join(dir_, f) for f in os.listdir(dir_)]
    for file in files:
        if resampling and "no_resampling" in file:
            continue
        elif not resampling and "no_resampling" not in file:
            continue
        with open(file, "rb") as f:
            result = pickle.load(f)
        results.append(result)
    return results


def compare(results) -> pd.DataFrame:
    stats = []
    for i, result in enumerate(results):
        stats.append(
            {
                f"Bad Debt Median": result.summary["Bad Debt Max"].median(),
                f"Bad Debt Mean": result.summary["Bad Debt Max"].mean(),
                f"Bad Debt p90": result.summary["Bad Debt Max"].quantile(0.90),
                f"Bad Debt p99": result.summary["Bad Debt Max"].quantile(0.99),
                f"Debt Liquidated Mean": result.summary["Debt Liquidated Max"].mean(),
                f"Debt Liquidated Median": result.summary[
                    "Debt Liquidated Max"
                ].median(),
                f"Debt Liquidated p90": result.summary["Debt Liquidated Max"].quantile(
                    0.90
                ),
                f"Debt Liquidated p99": result.summary["Debt Liquidated Max"].quantile(
                    0.99
                ),
                f"Borrower Loss Mean": result.summary["Borrower Loss Max"].mean(),
                f"Borrower Loss Median": result.summary["Borrower Loss Max"].median(),
                f"Borrower Loss p90": result.summary["Borrower Loss Max"].quantile(
                    0.90
                ),
                f"Borrower Loss p99": result.summary["Borrower Loss Max"].quantile(
                    0.99
                ),
                f"Total Debt Median": result.summary["Total Debt Mean"].median(),
                f"Total Debt p90": result.summary["Total Debt Mean"].quantile(0.90),
                f"Total Debt p99": result.summary["Total Debt Mean"].quantile(0.99),
                f"Total Crvusd Liquidity Median": result.summary[
                    "Total Crvusd Liquidity Min"
                ].median(),
                f"Total Crvusd Liquidity p90": result.summary[
                    "Total Crvusd Liquidity Min"
                ].quantile(0.90),
                f"Total Crvusd Liquidity p99": result.summary[
                    "Total Crvusd Liquidity Min"
                ].quantile(0.99),
            }
        )
    df = pd.DataFrame(stats).astype(int)
    return df


def compare_normed(results, metrics=["Bad Debt", "Debt Liquidated", "Borrower Loss"]):
    normed_stats = []
    for result in results:
        normed_stats_for_run = []
        for metric in metrics:
            normed_stats_for_metric = []
            for datum in result.data:
                sample = datum.df
                sample[f"{metric} Pct"] = (
                    sample[f"{metric}"] / sample["Total Debt"] * 100
                )
                normed_stats_for_metric.append(sample[f"{metric} Pct"].max())
            normed_stats_for_run.append(
                {
                    f"{metric} Pct Mean": np.mean(normed_stats_for_metric),
                    f"{metric} Pct Median": np.median(normed_stats_for_metric),
                    f"{metric} Pct p90": np.quantile(normed_stats_for_metric, 0.90),
                    f"{metric} Pct p99": np.quantile(normed_stats_for_metric, 0.99),
                    f"{metric} Pct MoM": mean_mom(np.array(normed_stats_for_metric)),
                }
            )
        normed_stats.append({k: v for d in normed_stats_for_run for k, v in d.items()})

    df = pd.DataFrame(normed_stats)
    return df

In [138]:
scenario = "high_volatility"
results = get_results(scenario, resampling=True)
df = compare(results)
df_normed = compare_normed(results)
display(df)
df_normed

,Bad Debt Median,Bad Debt Mean,Bad Debt p90,Bad Debt p99,Debt Liquidated Mean,Debt Liquidated Median,Debt Liquidated p90,Debt Liquidated p99,Borrower Loss Mean,Borrower Loss Median,Borrower Loss p90,Borrower Loss p99,Total Debt Median,Total Debt p90,Total Debt p99,Total Crvusd Liquidity Median,Total Crvusd Liquidity p90,Total Crvusd Liquidity p99
0,0,14696,11,188774,756741,105238,2424955,7804785,152804,38567,448838,1378624,125865959,155234545,183035350,50013539,64940643,76615874
1,0,15480,66,251798,283254,60172,508416,4921361,70225,27226,143289,1024291,125230538,156204586,185709664,51052492,64395846,79784020
2,0,10201,5,247450,312438,59103,605757,4471789,74897,27585,144120,932507,125437235,153198948,187775098,50102865,64083267,78379588
3,0,4198,5,93033,471828,67511,1142622,6730060,95422,27199,235092,1097864,124161858,154412525,179322803,49902534,64315007,76789582
4,0,5997,6,154888,374528,62398,809825,5837181,84800,26605,184919,954601,125960216,156396909,186553819,51040597,64887555,76905439


,Bad Debt Pct Mean,Bad Debt Pct Median,Bad Debt Pct p90,Bad Debt Pct p99,Bad Debt Pct MoM,Debt Liquidated Pct Mean,Debt Liquidated Pct Median,Debt Liquidated Pct p90,Debt Liquidated Pct p99,Debt Liquidated Pct MoM,Borrower Loss Pct Mean,Borrower Loss Pct Median,Borrower Loss Pct p90,Borrower Loss Pct p99,Borrower Loss Pct MoM
0,0.012277,0.0,0.000009,0.160725,0.004006,0.611228,0.083600,1.913428,6.584591,0.586320,0.123182,0.031095,0.352344,1.095990,0.119216
1,0.013153,0.0,0.000043,0.212217,0.005577,0.226402,0.047022,0.431768,3.476637,0.198667,0.055870,0.021902,0.115631,0.738130,0.051617
2,0.008588,0.0,0.000004,0.203785,0.004084,0.248194,0.046791,0.494097,3.636837,0.207141,0.059562,0.021896,0.114237,0.710886,0.054490
3,0.003592,0.0,0.000004,0.087698,0.002293,0.383708,0.053033,0.944902,5.364465,0.355029,0.077396,0.022703,0.186102,0.876417,0.073376
4,0.004567,0.0,0.000005,0.123658,0.002994,0.301147,0.049512,0.629688,4.498655,0.275936,0.067809,0.021194,0.145622,0.775986,0.063673


In [134]:
scenario = "high_volatility"
results = get_results(scenario, resampling=False)
df = compare(results)
df_normed = compare_normed(results)
display(df)
df_normed

,Bad Debt Median,Bad Debt Mean,Bad Debt p90,Bad Debt p99,Debt Liquidated Mean,Debt Liquidated Median,Debt Liquidated p90,Debt Liquidated p99,Borrower Loss Mean,Borrower Loss Median,Borrower Loss p90,Borrower Loss p99,Total Debt Median,Total Debt p90,Total Debt p99,Total Crvusd Liquidity Median,Total Crvusd Liquidity p90,Total Crvusd Liquidity p99
0,0,8194,0,126271,209012,186941,344022,857150,58348,54840,77435,172743,89335336,89438980,89452227,30810219,31441229,31501290
1,0,5234,0,100706,197536,160179,340033,964376,59034,50279,82924,218769,89349833,89419570,89456739,30617155,31391297,31499099
2,0,11960,34174,249936,231245,191911,378157,1055849,62416,54000,86353,207377,89331608,89419195,89441905,30622631,31443115,31501291
3,0,10867,33183,125243,204378,173396,369037,500117,63367,57062,80299,204168,89314058,89424145,89456065,30663406,31375397,31501290
4,0,11138,31747,249936,215138,195208,363823,531567,59653,56562,82468,148882,89320533,89423985,89442405,30633051,31451641,31498791


,Bad Debt Pct Mean,Bad Debt Pct Median,Bad Debt Pct p90,Bad Debt Pct p99,Bad Debt Pct MoM,Debt Liquidated Pct Mean,Debt Liquidated Pct Median,Debt Liquidated Pct p90,Debt Liquidated Pct p99,Debt Liquidated Pct MoM,Borrower Loss Pct Mean,Borrower Loss Pct Median,Borrower Loss Pct p90,Borrower Loss Pct p99,Borrower Loss Pct MoM
0,0.009172,4.628678e-07,4.672481e-07,0.141654,0.004647,0.235200,0.209633,0.387261,0.969954,0.222539,0.065591,0.061542,0.086976,0.195467,0.063725
1,0.005869,4.630407e-07,4.669456e-07,0.113325,0.003892,0.222335,0.179622,0.382113,1.098042,0.211390,0.066376,0.056415,0.093327,0.249090,0.064066
2,0.013414,4.628641e-07,3.847164e-02,0.279342,0.009056,0.260481,0.215251,0.425123,1.204395,0.248135,0.070254,0.060675,0.097287,0.236552,0.068695
3,0.012181,4.634545e-07,3.731881e-02,0.140120,0.010186,0.230040,0.194724,0.415162,0.571501,0.226640,0.071325,0.064033,0.090457,0.232892,0.069196
4,0.012465,4.632410e-07,3.552651e-02,0.279439,0.008037,0.242007,0.218987,0.409471,0.600112,0.236893,0.067070,0.063534,0.093039,0.168708,0.066041


In [56]:
scenario = "internal_crunch"
results = get_results(scenario, resampling=True)
df = compare(results)
df_normed = compare_normed(results)
display(df)
df_normed

,Bad Debt Median,Bad Debt Mean,Bad Debt p90,Bad Debt p99,Debt Liquidated Mean,Debt Liquidated Median,Debt Liquidated p90,Debt Liquidated p99,Borrower Loss Mean,Borrower Loss Median,Borrower Loss p90,Borrower Loss p99,Total Debt Median,Total Debt p90,Total Debt p99,Total Crvusd Liquidity Median,Total Crvusd Liquidity p90,Total Crvusd Liquidity p99
0,0,2670,0,108575,109797,0,10243,2945321,20392,1092,7101,506324,126503251,154919955,177228970,38660753,48565271,57432163
1,0,0,0,0,1388,0,0,43325,1148,367,3292,9266,125849496,160384226,172545331,39689297,51921823,59343669
2,0,0,0,0,3874,0,5119,59530,1929,715,4558,9568,126179473,160124114,180396512,40357687,50357129,63546074
3,0,0,0,2,277779,0,65811,2887923,48143,1544,14545,649604,129513619,153936692,193856952,39011849,51058084,66524041
4,0,339,0,347,137614,0,40894,3258966,27288,841,14902,590146,125800292,166967748,208139446,38579871,55064769,67895076


,Bad Debt Pct Mean,Bad Debt Pct Median,Bad Debt Pct p90,Bad Debt Pct p99,Bad Debt Pct >1%,Debt Liquidated Pct Mean,Debt Liquidated Pct Median,Debt Liquidated Pct p90,Debt Liquidated Pct p99,Debt Liquidated Pct >1%,Borrower Loss Pct Mean,Borrower Loss Pct Median,Borrower Loss Pct p90,Borrower Loss Pct p99,Borrower Loss Pct >1%
0,2.514674e-03,0.0,0.0,9.159246e-02,0.0,0.078140,0.0,0.009126,1.822675,3.0,0.014657,0.000756,0.005294,0.313102,0.0
1,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.001082,0.0,0.000000,0.032497,0.0,0.000897,0.000295,0.002701,0.007553,0.0
2,1.426857e-08,0.0,0.0,1.426857e-08,0.0,0.003339,0.0,0.004418,0.037445,0.0,0.001520,0.000604,0.003246,0.006970,0.0
3,1.205368e-07,0.0,0.0,2.265146e-06,0.0,0.222691,0.0,0.049407,2.168294,4.0,0.038532,0.000977,0.012151,0.486782,1.0
4,2.400631e-04,0.0,0.0,2.445098e-04,0.0,0.105628,0.0,0.035905,2.301581,3.0,0.020893,0.000618,0.011666,0.416784,0.0


In [57]:
scenario = "internal_crunch"
results = get_results(scenario, resampling=False)
df = compare(results)
df_normed = compare_normed(results)
display(df)
df_normed

,Bad Debt Median,Bad Debt Mean,Bad Debt p90,Bad Debt p99,Debt Liquidated Mean,Debt Liquidated Median,Debt Liquidated p90,Debt Liquidated p99,Borrower Loss Mean,Borrower Loss Median,Borrower Loss p90,Borrower Loss p99,Total Debt Median,Total Debt p90,Total Debt p99,Total Crvusd Liquidity Median,Total Crvusd Liquidity p90,Total Crvusd Liquidity p99
0,0,0,0,0,0,0,0,0,779,756,1123,1733,89474280,89474280,89474280,31346120,31489129,31499754
1,0,0,0,0,0,0,0,0,769,735,1147,1484,89474280,89474280,89474280,31282324,31487777,31501252
2,0,0,0,0,0,0,0,0,760,722,1179,1575,89474280,89474280,89474280,31429634,31501223,31501286
3,0,0,0,0,0,0,0,0,777,775,1140,1537,89474280,89474280,89474280,31394139,31498509,31500047
4,0,0,0,0,0,0,0,0,767,744,1152,1274,89474280,89474280,89474280,31416353,31500504,31501284


,Bad Debt Pct Mean,Bad Debt Pct Median,Bad Debt Pct p90,Bad Debt Pct p99,Bad Debt Pct >1%,Debt Liquidated Pct Mean,Debt Liquidated Pct Median,Debt Liquidated Pct p90,Debt Liquidated Pct p99,Debt Liquidated Pct >1%,Borrower Loss Pct Mean,Borrower Loss Pct Median,Borrower Loss Pct p90,Borrower Loss Pct p99,Borrower Loss Pct >1%
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000871,0.000846,0.001256,0.001938,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000861,0.000822,0.001282,0.001660,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000850,0.000807,0.001319,0.001761,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000869,0.000866,0.001275,0.001718,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000858,0.000832,0.001288,0.001424,0.0


In [136]:
scenario = "baseline"
results = get_results(scenario, resampling=True)
df = compare(results)
df_normed = compare_normed(results)
display(df)
df_normed

,Bad Debt Median,Bad Debt Mean,Bad Debt p90,Bad Debt p99,Debt Liquidated Mean,Debt Liquidated Median,Debt Liquidated p90,Debt Liquidated p99,Borrower Loss Mean,Borrower Loss Median,Borrower Loss p90,Borrower Loss p99,Total Debt Median,Total Debt p90,Total Debt p99,Total Crvusd Liquidity Median,Total Crvusd Liquidity p90,Total Crvusd Liquidity p99
0,0,92,0,94,198245,0,18032,4403102,33510,680,5644,645797,126342320,154227148,174594698,51390928,60825022,79141320
1,0,0,0,0,13777,0,2554,61627,3482,593,4143,15003,130097608,161060783,209497165,51933794,69947689,91516041
2,0,0,0,0,33405,0,0,63503,6377,452,3933,16952,130177216,159723315,178366423,53092503,64870907,81251327
3,0,648,0,651,115493,0,50723,2287020,25286,1090,17195,470339,128715552,157146184,178906499,53068117,66742335,75101091
4,0,17,0,27,75861,0,160,1790368,13048,632,4042,272645,131760236,158479500,194824494,53214675,67677253,81172761


,Bad Debt Pct Mean,Bad Debt Pct Median,Bad Debt Pct p90,Bad Debt Pct p99,Bad Debt Pct MoM,Debt Liquidated Pct Mean,Debt Liquidated Pct Median,Debt Liquidated Pct p90,Debt Liquidated Pct p99,Debt Liquidated Pct MoM,Borrower Loss Pct Mean,Borrower Loss Pct Median,Borrower Loss Pct p90,Borrower Loss Pct p99,Borrower Loss Pct MoM
0,6.500359e-05,0.0,0.0,6.683112e-05,0.0,0.142572,0.0,0.012945,3.357286,0.047707,0.024371,0.000553,0.004476,0.492358,0.009472
1,5.977222e-08,0.0,0.0,5.977222e-08,0.0,0.010168,0.0,0.002257,0.046847,0.000493,0.002567,0.000466,0.003194,0.009524,0.001086
2,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.027912,0.0,0.000000,0.051033,0.000078,0.005262,0.000349,0.003192,0.014616,0.001041
3,6.498445e-04,0.0,0.0,6.524535e-04,0.0,0.089399,0.0,0.037524,1.789384,0.033749,0.019639,0.000858,0.011271,0.367985,0.008976
4,1.367444e-05,0.0,0.0,2.030666e-05,0.0,0.057803,0.0,0.000119,1.626553,0.000800,0.009896,0.000497,0.003009,0.232498,0.001217


In [137]:
scenario = "baseline"
results = get_results(scenario, resampling=False)
df = compare(results)
df_normed = compare_normed(results)
display(df)
df_normed

,Bad Debt Median,Bad Debt Mean,Bad Debt p90,Bad Debt p99,Debt Liquidated Mean,Debt Liquidated Median,Debt Liquidated p90,Debt Liquidated p99,Borrower Loss Mean,Borrower Loss Median,Borrower Loss p90,Borrower Loss p99,Total Debt Median,Total Debt p90,Total Debt p99,Total Crvusd Liquidity Median,Total Crvusd Liquidity p90,Total Crvusd Liquidity p99
0,0,0,0,0,0,0,0,0,783,815,1166,1436,89474280,89474280,89474280,31397499,31498515,31501270
1,0,0,0,0,0,0,0,0,805,790,1206,1393,89474280,89474280,89474280,31298215,31490522,31501273
2,0,0,0,0,0,0,0,0,774,759,1176,1836,89474280,89474280,89474280,31415211,31501229,31501288
3,0,0,0,0,0,0,0,0,717,715,1069,1463,89474280,89474280,89474280,31391743,31498780,31500186
4,0,0,0,0,0,0,0,0,837,794,1222,1559,89474280,89474280,89474280,31437494,31499332,31499350


,Bad Debt Pct Mean,Bad Debt Pct Median,Bad Debt Pct p90,Bad Debt Pct p99,Bad Debt Pct MoM,Debt Liquidated Pct Mean,Debt Liquidated Pct Median,Debt Liquidated Pct p90,Debt Liquidated Pct p99,Debt Liquidated Pct MoM,Borrower Loss Pct Mean,Borrower Loss Pct Median,Borrower Loss Pct p90,Borrower Loss Pct p99,Borrower Loss Pct MoM
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000875,0.000911,0.001304,0.001606,0.000873
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000900,0.000884,0.001349,0.001558,0.000899
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000866,0.000849,0.001315,0.002052,0.000860
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000802,0.000800,0.001196,0.001636,0.000800
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000936,0.000888,0.001366,0.001743,0.000933
